In [2]:
import cv2
import mediapipe as mp
import autopy
import numpy as np

# Variables Declaration
width, height = 640, 480
frameR = 100
smoothing = 7

prev_x, prev_y = 0, 0
curr_x, curr_y = 0, 0

cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

screen_width, screen_height = autopy.screen.size()

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flip the video horizontally to remove the mirror effect
    imgRGB = cv2.flip(imgRGB, 1)

    results = hands.process(imgRGB)
    landmarkList = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmarkList.append([cx, cy])

            mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    if len(landmarkList) != 0:
        x1, y1 = landmarkList[8]
        x2, y2 = landmarkList[12]

        cv2.rectangle(img, (frameR, frameR), (width - frameR, height - frameR), (255, 0, 255), 2)

        if landmarkList[8] and landmarkList[12]:
            x3 = np.interp(x1, (frameR, width - frameR), (0, screen_width))
            y3 = np.interp(y1, (frameR, height - frameR), (0, screen_height))

            curr_x = prev_x + (x3 - prev_x) / smoothing
            curr_y = prev_y + (y3 - prev_y) / smoothing

            autopy.mouse.move(screen_width - int(curr_x), int(curr_y))
            cv2.circle(img, (x1, y1), 7, (255, 0, 255), cv2.FILLED)
            prev_x, prev_y = curr_x, curr_y

        # Detect two raised fingers (index and middle finger) for a left mouse click
        if landmarkList[8][1] < landmarkList[6][1] and landmarkList[12][1] < landmarkList[10][1]:
            autopy.mouse.click()

    cv2.imshow("AI Virtual Mouse", img)

    if cv2.waitKey(1) == 27:  # Exit when 'Esc' key is pressed
        break

cap.release()
cv2.destroyAllWindows()
